In [11]:
import numpy as np
import pandas as pd

## Loading data

In [12]:
#df_footprint = pd.read_parquet('F:/2024_Lightning_Analysis/FY-2E_data/data_extracted\\2018_part_0_small_ALL_data_with_lightning.parquet')
#df_station = pd.read_parquet('F:/2024_Lightning_Analysis/FY-2E_data/data_extracted\\around_station_small.parquet')
df_footprint = pd.read_parquet('F:/2024_Lightning_Analysis/FY-2E_data/data_extracted\\2018_part_3_of_6_ALL_2018_data_with_lightning.parquet')
df_station = pd.read_parquet('F:/2024_Lightning_Analysis/FY-2E_data/data_extracted\\around_station_2018_part_3_of_6_small.parquet')

In [13]:
print(df_footprint.shape)
print(df_station.shape)

(11852748, 25)
(137172, 25)


In [8]:
df_station['Month']

0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
1659    1.0
1660    1.0
1661    1.0
1662    1.0
1663    1.0
Name: Month, Length: 1664, dtype: float64

## Plot diurnal cycle

In [14]:
import plotly.graph_objs as go

def plot_diurnal_cycle_for_lightning(df_0=None, df_1=None, month=None):
    """
    Функция для построения суточного цикла разрядов молний для определённого месяца.
    Работает с одним или двумя наборами данных и строит графики на одной оси.
    
    Параметры:
        df_0 (pd.DataFrame, optional): Первый DataFrame с данными о молниях.
        df_1 (pd.DataFrame, optional): Второй DataFrame с данными о молниях.
        month (int): Номер месяца для фильтрации данных (1 - январь, 2 - февраль и т.д.).
    """
    # Убедимся, что месяц передан корректно
    if month is None or not (1 <= month <= 12):
        raise ValueError("Укажите корректный месяц от 1 до 12.")

    # Проверяем, что хотя бы один DataFrame передан
    if df_0 is None and df_1 is None:
        raise ValueError("Необходимо передать хотя бы один DataFrame (df_0 или df_1).")
    
    # Создаем пустой график
    fig = go.Figure()

    # Функция для нормировки данных
    def normalize_data(df, group_name):
        df_month = df[df['Month'] == month]
        counts_per_hour = df_month.groupby('Hour').size().reset_index(name=f'Number_of_Discharges_{group_name}')
        total_discharges = counts_per_hour[f'Number_of_Discharges_{group_name}'].sum()
        counts_per_hour[f'Number_of_Discharges_{group_name}'] /= total_discharges  # Нормировка на единицу
        return counts_per_hour

    # Обрабатываем первый набор данных, если передан
    if df_0 is not None:
        lightning_per_hour_0 = normalize_data(df_0, '0')

        # Добавляем кривую для df_0
        fig.add_trace(go.Scatter(
            x=lightning_per_hour_0['Hour'],
            y=lightning_per_hour_0['Number_of_Discharges_0'],
            mode='lines+markers',
            name=f'Набор данных 0 - Месяц {int(month)}',
            line=dict(color='blue')
        ))

    # Обрабатываем второй набор данных, если передан
    if df_1 is not None:
        lightning_per_hour_1 = normalize_data(df_1, '1')

        # Добавляем кривую для df_1
        fig.add_trace(go.Scatter(
            x=lightning_per_hour_1['Hour'],
            y=lightning_per_hour_1['Number_of_Discharges_1'],
            mode='lines+markers',
            name=f'Набор данных 1 - Месяц {int(month)}',
            line=dict(color='red')
        ))

    # Настраиваем внешний вид графика
    fig.update_layout(
        title=f'Нормированный суточный цикл разрядов молний - Месяц {int(month)}',
        xaxis_title='Местное время (Часы)',
        yaxis_title='Нормированное количество разрядов',
        xaxis=dict(tickmode='linear', dtick=1),  # Показываем каждый час
        template='plotly_dark'
    )

    # Показываем график
    fig.show()

In [17]:
# Пример использования:
# plot_diurnal_cycle_for_lightning(df_0, df_1, month=6)  # Построить графики для июня
# plot_diurnal_cycle_for_lightning(df_0, month=6)  # Построить график только для одного набора данных

plot_diurnal_cycle_for_lightning(df_footprint, df_station, 5)
